In [31]:
import csv
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import DBSCAN, KMeans

In [29]:
def clusters_to_csv(labels, types):
    '''
    Helper function to turn scikit-learn clusters into abbreviated CSVs
    '''
    for k in set(labels):
        class_members = [index[0] for index in np.argwhere(labels == k)]
        for index in class_members:
            print '%s,%s' % (int(k), types[index])

## Clustering documents

As we've discussed, the same principles that can be applied to clustering crime in two dimensions can also be applied to clustering doucments in much higher dimensional spaces. We'll demonstrate this concept using a selection of Jeb Bush's e-mails from his time serving as Florida's governor.

But first, we'll need to talk about what makes two documents "similar," which can be defined in a number of ways.

In [3]:
sample_docs = [
    'The quick brown fox jumped over the lazy dog',
    'The dog jumped over squirrel',
    'Four score and seven years ago'
]

We'll use those sample docs to start. Intuitively, you should be able to see that documents 0 and 1 have some similar elements ("dog," "jumped over," etc.) but document 2 is pretty different from the rest. Let's quantify that using two different distance measures: Euclidean and Cosine.

In [11]:
# First we'll vectorize our documents, as we did last week
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(sample_docs).toarray()
print features

[[0 0 0 1 1 0 1 1 1 1 1 0 0 0 2 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 0 1 2 0]
 [1 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1]]


## Euclidean distance

We'll start by looking at Euclidean distance.

Euclidean distance is what you probably most commonly think of when you think of distance. It's the shortest path between two points on a plane. If you take two points and draw a line between them, the length of that line is the Euclidean distance.

<img src="http://i.stack.imgur.com/w7kJP.png">

In [10]:
# We'll use a helpful scikit-learn function to calculate their pairwise distances, starting with Euclidean
euclidean_distances = pairwise_distances(features, metric='euclidean')
print euclidean_distances

[[ 0.          2.82842712  4.12310563]
 [ 2.82842712  0.          3.60555128]
 [ 4.12310563  3.60555128  0.        ]]


According to our Euclidean distance measure, document 0 and document 1 are 2.8 units apart, documents 0 and 2 are 4.1 units apart and documents 2 and 3 are 3.6 units apart. So this definitely captures the distances we're looking for. But in practice, there's another similarity measure that's more often used for looking at documents, known as cosine similarity.

## Cosine similarity

Unlike Euclidean distance, which looks at the absolute distance between points, cosine similarity accomplishes something similar by looking at the angle between them on a plane, like so:

<img src="https://engineering.aweber.com/wp-content/uploads/2013/02/4AUbj.png">

It is calculated via scikit-learn in a manner similar to Euclidean distance:

In [12]:
cosine_distances = pairwise_distances(features, metric='cosine')
print cosine_distances

[[  0.00000000e+00   4.30197118e-01   1.00000000e+00]
 [  4.30197118e-01   3.33066907e-16   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00  -2.22044605e-16]]


In practice, either one of these metrics can work for document similarity tasks. For now it's mostly important to know that there's more than one definition of similarity. Usually I start with cosine distance and test other metrics to see which work best for the task at hand.

## Clustering e-mails

After that little digression into distance metrics, we can now move on to clustering real documents -- in this case, subject lines from a selection of Jeb Bush's e-mails. Conveniently, we can use basically the same code as we used for the crime clustering example to accomplish this task.

In [36]:
emails = open('data/jeb_subjects.csv').read().split('\n')
print emails [:100]

['Budget Power Pt.', 'Re: Personal e-mail address', "RE: I'M BEGINNING TO WONDER IF I'VE BEEN DECEIVED....", 'Great to see you yesterday. . .', 'JAG', 'Council 100', 'Everglades Restoration Plan Science Caucus - Flow-ways', '"A good "', '', 'Response to JL Church', 'Invitation to Dominican Republic', 'Email typos', 'Talked to Sue.....', 'RE: Malathion Cover-up', '"FW: "', '', "FW: Open Gov't", "FW: Florida's Teachers", 'FW: Hi Speed Rail', 'FW: interested', 'FW: Welcome', 'FW: Phone Rate Increase', 'FW: Education - testing', 'Re: Invitation to Dominican Republic', 'FW: Natural Environment', 'FW: Hooray Governor Bush', '"RE: Vouchers, etc."', 'FOURTH DIMENSION', 'RE: Malathion Cover-up', 'Family Court Resource Assistance', 'RE: Approval', 'FW: (no subject)', 'Social Security alternative in Texas', 'Re: thanks', 'RE: Talked to Sue.....', 'Western Palm Beach County Farm Bureau', 'RE: FW: Vouchers', 'RE: thanks', "RE: I'M BEGINNING TO WONDER IF I'VE BEEN DECEIVED....", "Re: I'M BEGINNING T

In [32]:
# As usual, we'll vectorize our documents first
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(emails).toarray()

In [33]:
# Now we'll use k-means to try splitting them up into 20 groups
number_of_clusters = 20
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(features)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=20, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [34]:
print clusters_to_csv(kmeans.labels_, emails)

0,FW: Why rush kids into the classroom?/Tampa EDITORIAL
0,RE: smaller classroom sizes
0,FW: smaller classroom sizes
0,FW: smaller classroom sizes
0,Re: FW: Why rush kids into the classroom?/Tampa EDITORIAL
0,RE: FW: smaller classroom sizes
0,Re: FW: smaller classroom sizes
1,FOURTH DIMENSION
1,RE: FOURTH DIMENSION
1,FOURTH DIMENSION
1,Fourth Dimension Report for 01/15/99
1,Fourth Deminsion Report for 01/08/99
1,"Re: Telephone Report, Friday, February 26, 1999"
1,"Fourth Dimension report for February 19, 1999"
1,"Fourth Dimension report for February 11, 1999"
1,"FOURTH DIMENSION For February 05, 1999"
1,"Fourth Dimension report for March 5, 1999"
1,"Re: FW: Tuesday, 03/23/99, Phone Report"
1,"Fourth Dimension report for March 26, 1999"
1,RE: Weekly Report
1,"RE: Fourth Dimension report for March 19, 1999"
1,RE: Weekly Report
1,"Fourth Dimension report for March 19, 1999"
1,"Fourth Dimension report for April 08, 1999"
1,"Fourth Dimension report for April 01, 1999"
1,"THE NEW YORK TIMES M

You'll see that some clusters here make sense and others don't, which is partly a result of the size of the dataset we're using and the fact that subject lines are often too short to provide much meaningful clustering information. But you should also be able to see some clusters of documents that appear to make sense.

To try and get better results, let's raise the number of clusters in our dataset and also look at another vectorizing scheme known as TF-IDF, which is meant to minimize the value of words that occur frequently throughout the dataset.

In [35]:
# TF-IDF is invoked in the same way as a count vectorizer
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(emails).toarray()

# Let's try 50 clusters
number_of_clusters = 50
kmeans = KMeans(n_clusters=number_of_clusters)
kmeans.fit(features)

print clusters_to_csv(kmeans.labels_, emails)

0,FW: Hi Speed Rail
0,RE: hi
0,RE: hi there
0,hi there
0,Hi
0,RE: hi
0,hi
0,RE: HI
0,HI
0,RE: hi
0,hi
0,RE: hi from MArk
0,hi from MArk
0,RE: hi
0,hi
0,RE: hi
0,hi
0,FW: Hi
0,RE: Hi
0,Hi
0,RE: Hi
0,RE: Hi
0,Hi
1,Re: thanks
1,RE: thanks
1,thanks
1,thanks
1,thanks
1,Re: thanks
1,thanks
1,thanks
1,Re: thanks
1,Thanks...
1,RE: Many thanks...
1,FW: Much Thanks!
1,RE: Much Thanks!
1,FW: Thanks
1,FW: Thanks for your call
1,FW: Thanks
1,FW: Thanks
1,RE: Thanks
1,Thanks!
1,Thanks you
1,Re: thanks
1,RE: Thanks!
1,Thanks!
1,thanks for writng
1,thanks
1,THanks
1,"thanks "
1,"thanks "
1,Many Thanks!
1,thanks for writing
1,RE: Thanks...
1,Thanks...
1,Thanks
1,RE: Thanks
1,RE: Thanks!
1,Thanks
1,Re: thanks
1,thanks
1,Thanks
1,Thanks
1,thanks
1,RE: thanks
1,thanks
1,RE: Thanks!
1,RE: Thanks from Orlando
1,Thanks from Orlando
1,RE: Thanks!
1,RE: Thanks!
1,RE: Thanks!
1,FW: Thanks
1,Thanks!
1,Re:  thanks
1,many thanks
1,Re: thanks
1,RE: FW: Thanks
1,RE: FW: Thanks
1,Re: FW: Thanks
1,RE: Thanks
1,Thanks
